In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Let's do this

## Importing the libraries 


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt

# LABEL ENCODING

In [ ]:
BASE = "../input/tabular-playground-series-feb-2021"
test = pd.read_csv(BASE + '/test.csv')

train = pd.read_csv(BASE + '/train.csv')

sample_sub = pd.read_csv(BASE + '/sample_submission.csv')

In [ ]:
train = train.drop(['cat4', 'cont4'], axis = 1) 
test = test.drop(['cat4', 'cont4'], axis=1)

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
target = train['target'].values

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.describe()

In [ ]:
cat_features = columns[:9]
cat_features

In [ ]:
for feature in cat_features:
    le = LabelEncoder()
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    test[feature] = le.transform(test[feature])

In [ ]:
cont_features = columns[9:]
cont_features

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

train_features = train[cont_features]
test_features = test[cont_features]

ct = ColumnTransformer([
        ('somename', MinMaxScaler(), ['cont0', 'cont1', 'cont2', 'cont3', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13'])
    ], remainder='passthrough')
ft = ColumnTransformer([
        ('someothername', MinMaxScaler(), ['cont0', 'cont1', 'cont2', 'cont3','cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13'])
    ], remainder='passthrough')

train[cont_features] = ct.fit_transform(train_features)

test[cont_features] = ft.fit_transform(test_features)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.head()
train.describe()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.iloc[0:][columns]

In [ ]:
train_cat_cols = [f"cat{i}" for i in range(10) if i is not 4]
train_num_col = [f"cont{i}" for i in range(14) if i is not 4]

In [ ]:
train_num_col

# A bit of feature engineering 


# Let's get the xgboost model


In [ ]:
!pip install --upgrade xgboost
import xgboost as xgb
xgb.__version__

In [ ]:
xgb_params= {
        "objective": "reg:squarederror",
        "max_depth": 6,
        "learning_rate": 0.01,
        "colsample_bytree": 0.4,
        "subsample": 0.6,
        "reg_alpha" : 6,
        "min_child_weight": 100,
        "n_jobs": 2,
        "seed": 2021,
        'tree_method': "gpu_hist",
        "gpu_id": 0,
    }

In [ ]:
catBoost_params = {
    "n_estimator": 850,
    "tast_type" : "GPU",
    "loss_function": 'RMSE',
    "eval_metric": 'RMSE',
    "metric_period": 1000,
    "use_best_model": True,
    "random_seed": 2021
}

In [ ]:
lgbm_params = {
    "random_state": 2021,
    "metric": "rmse",
    "n_jobs": -1,
    "cat_feature": [x for x in range(len(train_cat_cols))],
    "early_stopping_round": 150,
    "reg_alpha": 6.147694913504962,
    "reg_lambda": 0.002457826062076097,
    "colsample_bytree": 0.3,
    "learning_rate": 0.01,
    "max_depth": 30,
    "num_leaves": 100,
    "min_child_samples": 275,
    "n_estimators": 1600000,
    "cat_smooth": 40.0,
    "max_bin": 512,
    "min_data_per_group": 100,
    "bagging_freq": 1,
    "bagging_fraction": 0.7,
    "cat_l2": 12.0,
}

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
train_oof = np.zeros((300000,))
light_trainPredict = np.zeros((300000,))
cat_trainPredict = np.zeros((300000,))

cat_preds = 0
light_preds = 0


test_preds = 0
train_oof.shape

In [ ]:
test_0 = test[columns]
test

In [ ]:
Test = xgb.DMatrix(test[columns])

In [ ]:
NUM_FOLDS = 10
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
        print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        train_target, val_target = target[train_ind], target[val_ind]
        
        ## we are going to use cat boost
        
        cat_model = CatBoostRegressor(learning_rate = 0.1,
                                      iterations = 2000,
                                      task_type="GPU",
                                      loss_function= 'RMSE',
                                      eval_metric='RMSE',
                                      metric_period=1000, 
                                      use_best_model=True,
                                      random_seed=42,
                                      allow_writing_files = False,
                                      od_type = 'Iter',
                                      bagging_temperature = 0.8,
                                      depth = 6,
                                      od_wait = 20,)
        cat_model.fit(
        train_df,
        train_target,
        eval_set=[(val_df, val_target)],
        plot=True,
        early_stopping_rounds=250
        )
        cat_valpredict = cat_model.predict(val_df)
        cat_testPredict = cat_model.predict(test_0)
        
        cat_trainPredict[val_ind] = cat_valpredict
        cat_preds += cat_testPredict/NUM_FOLDS
        print(mean_squared_error(cat_valpredict, val_target, squared=False))
        
        ## we are going to use lightboost as well
        
        light_model = LGBMRegressor(**lgbm_params ) 
        light_model.fit(
        train_df, 
        train_target,
        eval_set=[(val_df, val_target)],
        verbose=100,
    )
        light_valpredict = light_model.predict(val_df)
        light_testPredict = light_model.predict(test_0)
        
        light_trainPredict[val_ind] = light_valpredict
        light_preds += light_testPredict/NUM_FOLDS
        print(mean_squared_error(light_valpredict, val_target, squared=False))
        
        
        ## xgboost model with Dmatrix which is used for optimization
        
        train_df = xgb.DMatrix(train_df, label=train_target)
        val_df = xgb.DMatrix(val_df, label=val_target)
        
        model =  xgb.train(xgb_params, train_df, 3600)
        temp_oof = model.predict(val_df)
        temp_test = model.predict(Test)

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        print(mean_squared_error(temp_oof, val_target, squared=False))

In [ ]:
mean_squared_error(train_oof, target, squared=False)

In [ ]:
mean_squared_error(light_trainPredict, target, squared=False)

In [ ]:
mean_squared_error(cat_trainPredict, target, squared=False)

In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
train_model = [train_oof,light_trainPredict,cat_trainPredict]
test_model = [ test_preds,cat_preds, light_preds]

In [ ]:
np.save('light_oof', light_trainPredict)
np.save('cat_oof', cat_trainPredict)
np.save('light_test_preds', light_preds)
np.save('cat_test_preds', cat_preds)

# Let's see how the individual models perform and then we can make a ensemble according to that

## CV score for 

### XGBOOST = 0.8422094126465101

### CatBOOST = 0.8512973171285753

### lightBOOST = 0.84181677324358


## LeaderBoard Score...

In [ ]:
sample_sub['target'] = test_preds
sample_sub.to_csv('xgBoost.csv', index=False)




In [ ]:
sample_sub['target'] = light_preds
sample_sub.to_csv('light_sub.csv',index=False)

In [ ]:
sample_sub['target'] = cat_preds
sample_sub.to_csv('cat_sub.csv',index=False)